## 03. Kakao Local API

### 도로명주소 -> 위도, 경도

In [1]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]
filename

Saving kakaoapikey.txt to kakaoapikey.txt


'kakaoapikey.txt'

In [2]:
with open(filename) as f:
    api_key = f.read()

In [3]:
import requests
from urllib.parse import quote

In [4]:
addr = '서울특별시 중구 세종대로 110(태평로1가)'
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
url = f'{search_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%A4%91%EA%B5%AC%20%EC%84%B8%EC%A2%85%EB%8C%80%EB%A1%9C%20110%28%ED%83%9C%ED%8F%89%EB%A1%9C1%EA%B0%80%29'

In [5]:
# Authorization : KakaoAK {REST_API_KEY}
result = requests.get(
    url,
    headers={"Authorization": f'KakaoAK {api_key}'}
    ).json()
result

{'documents': [{'address': {'address_name': '서울 중구 태평로1가 31',
    'b_code': '1114010300',
    'h_code': '1114055000',
    'main_address_no': '31',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_h_name': '명동',
    'region_3depth_name': '태평로1가',
    'sub_address_no': '',
    'x': '126.977829174031',
    'y': '37.5663174209601'},
   'address_name': '서울 중구 세종대로 110',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 중구 세종대로 110',
    'building_name': '서울특별시청',
    'main_building_no': '110',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_name': '태평로1가',
    'road_name': '세종대로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.977829174031',
    'y': '37.5663174209601',
    'zone_no': '04524'},
   'x': '126.977829174031',
   'y': '37.5663174209601'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [8]:
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])
lat, lng

(37.5663174209601, 126.977829174031)

### 공공기관.csv에 위도, 경도 추가

In [9]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 공공기관.csv to 공공기관.csv


In [11]:
import pandas as pd
df = pd.read_csv(filename)
df

,공공기관,도로명주소
0,서울특별시청,서울특별시 중구 세종대로 110(태평로1가)
1,성북구청,서울특별시 성북구 보문로 168(삼선동5가)
2,길음2동주민센터,서울특별시 성북구 삼양로2길 70(길음동)


In [12]:
lng_list, lat_list = [], []
for i in df.index:
    url = f'{search_url}?query={quote(df.도로명주소[i])}'
    result = requests.get(
        url,
        headers={"Authorization": f'KakaoAK {api_key}'}
    ).json()
    lng = float(result['documents'][0]['x'])
    lat = float(result['documents'][0]['y'])
    lng_list.append(lng)
    lat_list.append(lat)

In [13]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,공공기관,도로명주소,위도,경도
0,서울특별시청,서울특별시 중구 세종대로 110(태평로1가),37.566317,126.977829
1,성북구청,서울특별시 성북구 보문로 168(삼선동5가),37.589456,127.016850
2,길음2동주민센터,서울특별시 성북구 삼양로2길 70(길음동),37.606807,127.027279


In [14]:
df.to_csv('공공기관2.csv', index=False)

### 지도 위에 공공기관 표시

In [16]:
import folium
map = folium.Map(location = [df.위도.mean(), df.경도.mean()], zoom_start=12)
for i in df.index:
    folium.Circle(
        radius = 300,
        location = [df.위도[i], df.경도[i]],
        popup = folium.Popup(df.도로명주소[i], max_width=200),
        tooltip = df.공공기관[i],
        color = '#3186CC',
        fill = True,
        fill_color = '#3186CC'
    ).add_to(map)
title = '<h3 align = "center" style = "font-size:20px">공공기관 안내</h3>'
map.get_root().html.add_child(folium.Element(title))
map